## Verstehen von Q_n, Q_p und Q_Li als Inputs für die Bestimmung von den minimalen und maximalen stöchiometrischen Grenzen
Die Domänengröße `Q` hängt direkt mit der zyklierbarem Lithium-Ionen in dem Modell zusammen und dient als Grundlage zur Festlegung der maximalen und minimalen Stöchiometrien in den Elektroden.

---
### Fehlerbild aus "C_rate_experiment (24.01.2023)" - Auszug
Derzeit (24.01.2023) funktioniert der Prozess zum Setzen des anfänglichen Ladezustands nicht. Der Fehler deutet daraufhin, dass das die Bestimmung der maximalen und minimalen Stöchiometrien für die Zelle nicht funktionieren.

* pybamm.Simulation(kwargs).solve(initial_soc=soc_ini) - soc_ini = [0,1]
    * pybamm.simulation.set_initial_soc
        * pybamm.parameter_values.set_initial_stoichiometries
            * pybamm.lithium_ion.get_initial_stoichiometries -> esoh_solver.get_initial_stoichiometries
                * esoh_solver.get_min_max_stoichiometries
                    * <span style="color:red">**Was ist Q???**</span>.
---


### Berechnung von Q in PyBaMM
Die Berechnung von Q erfolgt insgesamt für die Zelle (oder ist hier der Elektrolyt gemeint?) und für die einzelnen Aktivamterialien.

in `pybamm.lihtium_ion_parameters` Zeile 138f für "Total Lithium":
    
    self.n_Li_particles_init = self.n.n_Li_init + self.p.n_Li_init
    self.n_Li_init = self.n_Li_particles_init + self.n_Li_e_init
    self.Q_Li_particles_init = self.n_Li_particles_init * self.F / 3600
    self.Q_Li_init = self.n_Li_init * self.F / 3600 

in `pybamm.lihtium_ion_parameters` Zeile 798f für **Partikel-Domänen** (primary material): 

    self.n_Li_init = eps_c_init_av * self.c_max * self.domain_param.L * main.A_cc
    self.Q_Li_init = self.n_Li_init * main.F / 3600

Das anfängliche gesamte Lithium inventory `self.n_Li_init` ist abhängig von der gesamten Lithium-Ionen Menge in den Partikeln `self.n_Li_particles_init` und dem Elektrolyten `self.n_Li_e_init` abhängig.

### Was ist "n"?
Die Berechnung von Q ist direkt mit der Variable `n`, der Faraday-Konstante `F` und dem Faktor 3600 (Zeit?) verbunden.

Schauen wir dazu eine Beispielrechnung für `n_Li_e_init` an. Diese wird nach `pybamm.lihtium_ion_parameters` Zeile 135f bestimmt:

    c_e_av_init = pybamm.xyz_average(self.epsilon_init) * self.c_e_typ
    self.n_Li_e_init = c_e_av_init * self.L_x * self.A_cc

Die erste Zeile multipliziert dabei ie nach der Dicke (P2D: x-Dimension) gewichtete Mittel der Porosität der einzelnen Zellkomponenten (Elektroden + Separator) im Modell. In der folgenden Zeilen wird daraufhin die Menge die gesamte Menge Lithium Ionen berechnet, der sich durch den Elektrolyten in dem aktiven Zellvolumen befindet.

Analog dazu wird auch die Stoffmengen der Lithium in den Aktivmaterialien bestimmt. Zunächst wird dazu die mittlere Konzentration von Lithium Ionen in dem Aktivmaterial berechnet, mit dem Aktivmaterialanteil multipliziert und dann über das gesamte Aktivmaterialvolumen gemittelt. Allerdings ist die anfängliche Konzentration über die maximale Konzentration in der in dem Aktivmaterial der Domäne normiert (Zeile 793).

In `pybamm.lihtium_ion_parameters` für Partikel-Domänen (primary material) - Zeile 793f:

    self.c_init = self.c_init_dimensional / self.c_max
    [...]
    eps_c_init_av = pybamm.xyz_average(
    self.epsilon_s * pybamm.r_average(self.c_init)
    )

Die gesamte Menge an Lithium Ionen wird folgend analog wie bei dem Elektrolyten wird die mittlere Konzetration in dem geometrischen Volumen der Aktivmaterialphase multipliziert. Zusätzlich wird noch mit der maximalen Konzentration multipliziert, um die vorherige Normierung wieder aufzuheben. 

In `pybamm.lihtium_ion_parameters` für Partikel-Domänen (primary material) - Zeile 798:

    self.n_Li_init = eps_c_init_av * self.c_max * self.domain_param.L * main.A_cc  
